In [6]:
%pip install xgboost
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import sys

import shutil
import random
import warnings

warnings.filterwarnings("ignore") # , category=UserWarning

import numpy as np
import pandas as pd

from decimal import *
from pathlib import Path
import glob

from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm
# from stepwise_regression import step_reg
import xgboost as xgb
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.cluster import hierarchy as hc

def cluster_columns(df, figsize=(10,6), font_size=12):
    corr = np.round(scipy.stats.spearmanr(df).correlation, 4)
    corr_condensed = hc.distance.squareform(1-corr)
    z = hc.linkage(corr_condensed, method='average')
    fig = plt.figure(figsize=figsize)
    hc.dendrogram(z, labels=df.columns, orientation='left', leaf_font_size=font_size)
    plt.show()

In [8]:
submission=pd.read_csv('../data/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv')
print(submission.shape)
submission.head()

(64, 3)


,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,0,0
1,3342_0_updrs_1_plus_6_months,0,0
2,3342_0_updrs_1_plus_12_months,0,0
3,3342_0_updrs_1_plus_24_months,0,0
4,3342_0_updrs_2_plus_0_months,0,0


In [9]:
test_clinical=pd.read_csv('../data/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv')
print(test_clinical.shape)
test_clinical.head()

(16, 6)


,visit_id,visit_month,patient_id,updrs_test,row_id,group_key
0,3342_0,0,3342,updrs_1,3342_0_updrs_1,0
1,3342_0,0,3342,updrs_2,3342_0_updrs_2,0
2,3342_0,0,3342,updrs_3,3342_0_updrs_3,0
3,3342_0,0,3342,updrs_4,3342_0_updrs_4,0
4,50423_0,0,50423,updrs_1,50423_0_updrs_1,0


In [10]:
train_clinical=pd.read_csv('../data/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
#pd.options.display.float_format = '{:,.0f}'.format
print(train_clinical.shape)
train_clinical.head()

(2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [11]:
train_clinical_drop=train_clinical.drop('upd23b_clinical_state_on_medication',axis=1)
train_clinical_drop.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
0,55_0,55,0,10.0,6.0,15.0,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0
4,55_12,55,12,10.0,10.0,41.0,0.0


In [13]:
test_proteins=pd.read_csv('../data/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv')
print(test_proteins.shape)
test_proteins.head()

(453, 6)


,visit_id,visit_month,patient_id,UniProt,NPX,group_key
0,50423_0,0,50423,O00391,33127.90,0
1,50423_0,0,50423,O00533,490742.00,0
2,50423_0,0,50423,O00584,43615.30,0
3,50423_0,0,50423,O14773,16486.60,0
4,50423_0,0,50423,O14791,2882.42,0


In [14]:
test_proteins_role=test_proteins
test_proteins_role[['role']]='Test'
test_proteins_role.head()

,visit_id,visit_month,patient_id,UniProt,NPX,group_key,role
0,50423_0,0,50423,O00391,33127.90,0,Test
1,50423_0,0,50423,O00533,490742.00,0,Test
2,50423_0,0,50423,O00584,43615.30,0,Test
3,50423_0,0,50423,O14773,16486.60,0,Test
4,50423_0,0,50423,O14791,2882.42,0,Test


In [15]:
test_proteins_scale=test_proteins.drop(['group_key'],axis=1)
test_proteins_scale.head()

,visit_id,visit_month,patient_id,UniProt,NPX,role
0,50423_0,0,50423,O00391,33127.90,Test
1,50423_0,0,50423,O00533,490742.00,Test
2,50423_0,0,50423,O00584,43615.30,Test
3,50423_0,0,50423,O14773,16486.60,Test
4,50423_0,0,50423,O14791,2882.42,Test


In [16]:
train_proteins=pd.read_csv('../data/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
train_proteins.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [17]:
train_proteins_role=train_proteins
train_proteins_role['group_key']=0
train_proteins_role['role']='Train'

train_proteins_role['visit_month']=train_proteins_role['visit_month'].astype(str)
train_proteins_role['patient_id']=train_proteins_role['patient_id'].astype(str)

train_proteins_role.head()

,visit_id,visit_month,patient_id,UniProt,NPX,group_key,role
0,55_0,0,55,O00391,11254.3,0,Train
1,55_0,0,55,O00533,732430.0,0,Train
2,55_0,0,55,O00584,39585.8,0,Train
3,55_0,0,55,O14498,41526.9,0,Train
4,55_0,0,55,O14773,31238.0,0,Train


In [19]:
test_peptides=pd.read_csv('../data/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv')
print(test_peptides.shape)

test_peptides_role=test_peptides
test_peptides_role[['role']]='Test'
test_peptides_role.head()

(2057, 7)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,group_key,role
0,50423_0,0,50423,O00391,AHFSPSNIILDFPAAGSAAR,22226.3,0,Test
1,50423_0,0,50423,O00391,NEQEQPLGQWHLS,10901.6,0,Test
2,50423_0,0,50423,O00533,GNPEPTFSWTK,51499.4,0,Test
3,50423_0,0,50423,O00533,IEIPSSVQQVPTIIK,125492.0,0,Test
4,50423_0,0,50423,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,23174.2,0,Test


In [21]:
train_peptides=pd.read_csv('../data/amp-parkinsons-disease-progression-prediction/train_peptides.csv')

train_peptides_role=train_peptides
train_peptides_role[['role']]='Train'
train_peptides_role.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,role
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3,Train
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0,Train
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0,Train
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9,Train
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7,Train


In [22]:
train_peptides.groupby('Peptide').size()

Peptide
AADDTWEPFASGK                                       1069
AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K    1049
AAFTEC(UniMod_4)C(UniMod_4)QAADK                    1107
AANEVSSADVK                                          909
AATGEC(UniMod_4)TATVGKR                              914
                                                    ... 
YVNKEIQNAVNGVK                                      1111
YWGVASFLQK                                          1089
YYC(UniMod_4)FQGNQFLR                               1105
YYTYLIMNK                                           1030
YYWGGQYTWDMAK                                        865
Length: 968, dtype: int64

In [23]:
train_clinical.loc[train_clinical['updrs_1'].between(0, 10, 'both'), '1 Dl (non-m)'] = 'Mild'
train_clinical.loc[train_clinical['updrs_1'].between(10, 21, 'right'), '1 Dl (non-m)'] = 'Moderate'
train_clinical.loc[train_clinical['updrs_1'].between(21, 52, 'right'), '1 Dl (non-m)'] = 'Severe'

train_clinical.loc[train_clinical['updrs_2'].between(0, 12, 'both'), '2 Dl (mtr)'] = 'Mild'
train_clinical.loc[train_clinical['updrs_2'].between(12,29, 'right'), '2 Dl (mtr)'] = 'Moderate'
train_clinical.loc[train_clinical['updrs_2'].between(29, 52, 'right'), '2 Dl (mtr)'] = 'Severe'

train_clinical.loc[train_clinical['updrs_3'].between(0, 32, 'both'), '3 Mtr exam'] = 'Mild'
train_clinical.loc[train_clinical['updrs_3'].between(32,58, 'right'), '3 Mtr exam'] = 'Moderate'
train_clinical.loc[train_clinical['updrs_3'].between(58, 132, 'right'), '3 Mtr exam'] = 'Severe'

train_clinical.loc[train_clinical['updrs_4'].between(0, 4, 'both'), '4 Mtr comp'] = 'Mild'
train_clinical.loc[train_clinical['updrs_4'].between(4,12, 'right'), '4 Mtr comp'] = 'Moderate'
train_clinical.loc[train_clinical['updrs_4'].between(12, 24, 'right'), '4 Mtr comp'] = 'Severe'

train_clinical['counter']=1

In [24]:
train_clinical=train_clinical.copy()
train_clinical['updrs_sum']=train_clinical[['updrs_1','updrs_2','updrs_3','updrs_4']].sum(axis=1)
train_clinical.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,1 Dl (non-m),2 Dl (mtr),3 Mtr exam,4 Mtr comp,counter,updrs_sum
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,Mild,Mild,Mild,NaN,1,31.0
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,Mild,Mild,Mild,NaN,1,42.0
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,Mild,Mild,Moderate,NaN,1,52.0
3,55_9,55,9,8.0,9.0,30.0,0.0,On,Mild,Mild,Mild,Mild,1,47.0
4,55_12,55,12,10.0,10.0,41.0,0.0,On,Mild,Mild,Moderate,Mild,1,61.0


In [25]:
train_clinical.sort_values(['patient_id','visit_month'], inplace=True)
train_clinical['diffs_1'] = train_clinical.groupby(['patient_id'])['updrs_1'].transform(lambda x: x.diff())
train_clinical['diffs_2'] = train_clinical.groupby(['patient_id'])['updrs_2'].transform(lambda x: x.diff())
train_clinical['diffs_3'] = train_clinical.groupby(['patient_id'])['updrs_3'].transform(lambda x: x.diff())
train_clinical['diffs_4'] = train_clinical.groupby(['patient_id'])['updrs_4'].transform(lambda x: x.diff())
train_clinical['diffs_sum'] = train_clinical.groupby(['patient_id'])['updrs_sum'].transform(lambda x: x.diff())

train_clinical=train_clinical[['visit_id', 'patient_id','visit_month',
                               'updrs_1','1 Dl (non-m)','diffs_1',
                               'updrs_2','2 Dl (mtr)','diffs_2',
                               'updrs_3','3 Mtr exam','diffs_3',
                               'updrs_4','4 Mtr comp','diffs_4',
                               'updrs_sum','diffs_sum',
                               'counter','upd23b_clinical_state_on_medication']]

train_clinical.head()

,visit_id,patient_id,visit_month,updrs_1,1 Dl (non-m),diffs_1,updrs_2,2 Dl (mtr),diffs_2,updrs_3,3 Mtr exam,diffs_3,updrs_4,4 Mtr comp,diffs_4,updrs_sum,diffs_sum,counter,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,Mild,NaN,6.0,Mild,NaN,15.0,Mild,NaN,NaN,NaN,NaN,31.0,NaN,1,NaN
1,55_3,55,3,10.0,Mild,0.0,7.0,Mild,1.0,25.0,Mild,10.0,NaN,NaN,NaN,42.0,11.0,1,NaN
2,55_6,55,6,8.0,Mild,-2.0,10.0,Mild,3.0,34.0,Moderate,9.0,NaN,NaN,NaN,52.0,10.0,1,NaN
3,55_9,55,9,8.0,Mild,0.0,9.0,Mild,-1.0,30.0,Mild,-4.0,0.0,Mild,NaN,47.0,-5.0,1,On
4,55_12,55,12,10.0,Mild,2.0,10.0,Mild,1.0,41.0,Moderate,11.0,0.0,Mild,0.0,61.0,14.0,1,On


In [27]:
proteins_wide = pd.pivot(train_proteins, index=['visit_id','visit_month','patient_id','group_key','role'],columns='UniProt',values='NPX')
proteins_wide = proteins_wide.rename_axis(None, axis=1).reset_index()
proteins_wide = proteins_wide.replace('Train', np.nan)  # replace non-numeric values with np.nan
proteins_wide = proteins_wide.fillna(proteins_wide.median())
proteins_wide = proteins_wide.drop(['group_key','role'],axis=1)

train_clinical_drop['visit_month'] = train_clinical_drop['visit_month'].astype(str)
train_clinical_drop['patient_id'] = train_clinical_drop['patient_id'].astype(str)

train_clinical_proteins = pd.merge(train_clinical_drop,proteins_wide,how='left',on=['visit_id','patient_id','visit_month'])
train_clinical_proteins.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O00391,O00533,O00584,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,55_0,55,0,10.0,6.0,15.0,NaN,11254.3,732430.0,39585.8,...,365475.0,35528.0,97005.6,23122.5,60912.6,408698.0,67571.7,29758.8,23833.7,18953.5
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,13163.6,630465.0,35220.8,...,405676.0,30332.6,109174.0,23499.8,51655.8,369870.0,67571.7,22935.2,17722.5,16642.7
3,55_9,55,9,8.0,9.0,30.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55_12,55,12,10.0,10.0,41.0,0.0,15257.6,815083.0,41650.9,...,303953.0,43026.2,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9


In [28]:
df_updrs_1=train_clinical_proteins
df_updrs_1=df_updrs_1.dropna(subset=['Q99435', 'Q99674', 'Q99683', 'Q99829', 'Q99832'])
df_updrs_1=df_updrs_1.drop(['visit_id','patient_id','visit_month','updrs_2','updrs_3','updrs_4'],axis=1)
df_updrs_1=df_updrs_1.fillna(proteins_wide.median())
print(df_updrs_1.shape)
df_updrs_1.head()

(1068, 228)


,updrs_1,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,10.0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,62898.2,333376.0,...,365475.0,35528.0,97005.6,23122.5,60912.6,408698.0,67571.7,29758.8,23833.7,18953.5
2,8.0,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,62567.5,277833.0,...,405676.0,30332.6,109174.0,23499.8,51655.8,369870.0,67571.7,22935.2,17722.5,16642.7
4,10.0,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,66963.1,332401.0,...,303953.0,43026.2,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9
8,17.0,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,67588.6,317490.0,...,303597.0,48188.4,109794.0,23930.6,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1
15,8.0,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,43423.1,165524.0,...,253373.0,27431.8,93796.7,17450.9,21299.1,306621.0,82335.5,24018.7,18939.5,15251.2


In [29]:
X=df_updrs_1.drop('updrs_1', axis=1)
y=df_updrs_1.iloc[:,0]

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=15
)

# Scale train and test sets with StandardScaler
#X_train_std = StandardScaler().fit_transform(X_train)
#X_test_std = StandardScaler().fit_transform(X_test)

# Fix the dimensions of the target array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

m = XGBRegressor()
m.fit(X, y)

feat_dict={}
for col, val in sorted(zip(X_train.columns, m.feature_importances_), key=lambda x:x[1],reverse=True):
    feat_dict[col]=val
    
updrs_1_xgb = pd.DataFrame({'Feature':feat_dict.keys(),'Importance':feat_dict.values()})

updrs_1_xgb.to_csv('updrs_1_xgb.csv')

In [30]:
# Find cumulative importance and create list of columns to model

updrs_1_xgb['Cum_importance']=updrs_1_xgb['Importance'].cumsum()
updrs_1_xgb

updrs_1_xgb_keep=updrs_1_xgb.drop(updrs_1_xgb[updrs_1_xgb.Cum_importance >=0.80].index)
updrs_1_xgb_keep

updrs_1_xgb_keep_list = updrs_1_xgb_keep.Feature.values.tolist()

In [31]:
# Apply updrs_1_xgb_keep_list to test_proteins as this df doesn't include all the proteins of interest

test_proteins_wide = pd.pivot(test_proteins,index=['visit_id','visit_month','patient_id','group_key','role'],columns='UniProt',values='NPX')
test_proteins_wide=test_proteins_wide.rename_axis(None, axis=1).reset_index()
test_proteins_wide=test_proteins_wide.drop(['visit_id','visit_month','patient_id','group_key','role'], axis=1)
test_proteins_wide=test_proteins_wide[test_proteins_wide.columns.intersection(updrs_1_xgb_keep_list)]
test_proteins_wide=test_proteins_wide.replace(np.nan,0)
test_proteins_wide.head()

,O14773,O75144,P00736,P00746,P00748,P01009,P01011,P01031,P01033,P01034,...,Q8N2S1,Q8NE71,Q92520,Q96BZ4,Q96S96,Q99969,Q9NQ79,Q9UNU6,Q9Y646,Q9Y6R7
0,14959.2,160410.0,115974.0,107976.0,61713.9,16205500.0,1708870.0,11004.3,0.0,28752900.0,...,6057.78,0.0,1029960.0,12589.8,34384.9,47999.1,46411.8,25117.5,51473.3,14624.5
1,16486.6,119277.0,106727.0,146538.0,114337.0,14412800.0,2065690.0,10003.1,105886.0,29182200.0,...,9206.85,14768.7,602411.0,12505.2,26700.6,78596.4,35489.7,20925.2,38865.3,42920.4


In [32]:
test_proteins_long=test_proteins_wide.melt(ignore_index=False).reset_index()
test_proteins_long.head()
test_proteins_list = test_proteins_long.variable.values.tolist()
test_proteins_list=pd.unique(test_proteins_list).tolist()

In [33]:
test_proteins_list.insert(0,'updrs_1')
df_updrs_1_model=df_updrs_1[df_updrs_1.columns.intersection(test_proteins_list)]
print(df_updrs_1_model.shape)
df_updrs_1_model.head()

(1068, 89)


,updrs_1,O14773,O75144,P00736,P00746,P00748,P01009,P01011,P01031,P01033,...,Q8N2S1,Q8NE71,Q92520,Q96BZ4,Q96S96,Q99969,Q9NQ79,Q9UNU6,Q9Y646,Q9Y6R7
0,10.0,31238.00,98746.3,109541.0,111619.0,71835.4,14415900.0,2025890.0,13033.3,117641.0,...,10802.80,36202.0,982965.0,13189.30,34868.9,22021.6,35528.0,29758.8,23833.7,18953.5
2,8.0,26219.90,95184.0,105767.0,95825.1,44460.6,13330800.0,2060930.0,12149.3,128553.0,...,11604.60,29426.9,860236.0,12739.30,33386.8,23109.0,30332.6,22935.2,17722.5,16642.7
4,10.0,30703.60,93159.8,112909.0,127383.0,46942.3,13225500.0,2015710.0,14103.5,138784.0,...,10101.10,34246.7,969345.0,11830.00,39880.0,22730.6,43026.2,29193.4,28536.1,19290.9
8,17.0,33577.60,96991.1,98673.4,57012.0,73678.9,13855000.0,2221880.0,14416.0,167034.0,...,11812.70,24592.0,910437.0,14266.20,40086.4,23715.0,48188.4,31732.6,22186.5,21717.1
15,8.0,6173.58,49192.5,18608.6,84433.1,101312.0,13667800.0,2188150.0,14947.6,38014.4,...,6483.25,20903.1,639154.0,6911.97,32527.9,13060.3,27431.8,24018.7,18939.5,15251.2


In [34]:
X=df_updrs_1_model.drop('updrs_1', axis=1)
y=df_updrs_1_model.iloc[:,0]

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=30
)

# Scale train and test sets with StandardScaler
#X_train_std = StandardScaler().fit_transform(X_train)
#X_test_std = StandardScaler().fit_transform(X_test)

# Fix the dimensions of the target array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

In [35]:
m = XGBRegressor()
m.fit(X_train, y_train)

feat_dict={}
for col, val in sorted(zip(X_train.columns, m.feature_importances_), key=lambda x:x[1],reverse=True):
    feat_dict[col]=val

In [36]:
updrs_1_feat_xgb_model = pd.DataFrame({'Feature':feat_dict.keys(),'Importance':feat_dict.values()})
updrs_1_feat_xgb_model.to_csv('updrs_1_xgb_model.csv')

In [37]:
y_pred = m.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

MSE: 26.38
RMSE: 5.14


In [38]:
len(y_pred)

321

In [39]:
updrs_1_array = m.predict(test_proteins_wide)
#mse = mean_squared_error(y_test, y_pred)
#print("MSE: %.2f" % mse)
#print("RMSE: %.2f" % (mse**(1/2.0)))

In [40]:
updrs_1_array

array([4.2874784, 3.1299646], dtype=float32)

In [41]:
submission

,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,0,0
1,3342_0_updrs_1_plus_6_months,0,0
2,3342_0_updrs_1_plus_12_months,0,0
3,3342_0_updrs_1_plus_24_months,0,0
4,3342_0_updrs_2_plus_0_months,0,0
...,...,...,...
59,50423_6_updrs_3_plus_24_months,0,6
60,50423_6_updrs_4_plus_0_months,0,6
61,50423_6_updrs_4_plus_6_months,0,6
62,50423_6_updrs_4_plus_12_months,0,6


In [42]:
updrs_1_pred = pd.DataFrame(data=updrs_1_array,
                           columns=['Score'])
updrs_1_pred

updrs_1_pred.to_csv('updrs_1_pred.csv')
submission.at[1,'rating']= updrs_1_array[0]
submission.at[16,'rating']= updrs_1_array[1]

In [43]:
df_updrs_2=train_clinical_proteins
df_updrs_2=df_updrs_2.dropna(subset=['Q99435', 'Q99674', 'Q99683', 'Q99829', 'Q99832'])
df_updrs_2=df_updrs_2.drop(['visit_id','patient_id','visit_month','updrs_1','updrs_3','updrs_4'],axis=1)
df_updrs_2=df_updrs_2.fillna(proteins_wide.median())
print(df_updrs_2.shape)
df_updrs_2.head()

(1068, 228)


,updrs_2,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,6.0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,62898.2,333376.0,...,365475.0,35528.0,97005.6,23122.5,60912.6,408698.0,67571.7,29758.8,23833.7,18953.5
2,10.0,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,62567.5,277833.0,...,405676.0,30332.6,109174.0,23499.8,51655.8,369870.0,67571.7,22935.2,17722.5,16642.7
4,10.0,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,66963.1,332401.0,...,303953.0,43026.2,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9
8,18.0,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,67588.6,317490.0,...,303597.0,48188.4,109794.0,23930.6,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1
15,2.0,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,43423.1,165524.0,...,253373.0,27431.8,93796.7,17450.9,21299.1,306621.0,82335.5,24018.7,18939.5,15251.2


In [44]:
X=df_updrs_2.drop('updrs_2', axis=1)
y=df_updrs_2.iloc[:,0]

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=15
)

# Scale train and test sets with StandardScaler
#X_train_std = StandardScaler().fit_transform(X_train)
#X_test_std = StandardScaler().fit_transform(X_test)

# Fix the dimensions of the target array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

m = XGBRegressor()
m.fit(X, y)

feat_dict={}
for col, val in sorted(zip(X_train.columns, m.feature_importances_), key=lambda x:x[1],reverse=True):
    feat_dict[col]=val
    
updrs_2_xgb = pd.DataFrame({'Feature':feat_dict.keys(),'Importance':feat_dict.values()})

updrs_2_xgb.to_csv('updrs_2_xgb.csv')

In [45]:
# Find cumulative importance and create list of columns to model

updrs_2_xgb['Cum_importance']=updrs_2_xgb['Importance'].cumsum()
updrs_2_xgb

updrs_2_xgb_keep=updrs_2_xgb.drop(updrs_2_xgb[updrs_2_xgb.Cum_importance >=0.80].index)
updrs_2_xgb_keep

updrs_2_xgb_keep_list = updrs_2_xgb_keep.Feature.values.tolist()

In [46]:
# Apply updrs_2_xgb_keep_list to test_proteins as this df doesn't include all the proteins of interest

test_proteins_wide = pd.pivot(test_proteins,index=['visit_id','visit_month','patient_id','group_key','role'],columns='UniProt',values='NPX')
test_proteins_wide=test_proteins_wide.rename_axis(None, axis=1).reset_index()
test_proteins_wide=test_proteins_wide.drop(['visit_id','visit_month','patient_id','group_key','role'], axis=1)
test_proteins_wide=test_proteins_wide[test_proteins_wide.columns.intersection(updrs_2_xgb_keep_list)]
test_proteins_wide=test_proteins_wide.replace(np.nan,0)
print(test_proteins_wide.shape)
test_proteins_wide.head()

(2, 92)


,O14773,O94919,P00736,P00738,P00748,P01009,P01011,P01019,P01033,P01034,...,Q6UXD5,Q7Z5P9,Q92823,Q92876,Q99969,Q9BY67,Q9UBR2,Q9UKV8,Q9UNU6,Q9Y6R7
0,14959.2,69440.7,115974.0,5200440.0,61713.9,16205500.0,1708870.0,2324710.0,0.0,28752900.0,...,0.0,474377.0,240527.0,1364170.0,47999.1,64253.8,16836.0,64411.5,25117.5,14624.5
1,16486.6,53987.7,106727.0,563640.0,114337.0,14412800.0,2065690.0,1669670.0,105886.0,29182200.0,...,15938.2,702272.0,157497.0,1457830.0,78596.4,29437.6,22868.1,72595.0,20925.2,42920.4


In [47]:
test_proteins_long=test_proteins_wide.melt(ignore_index=False).reset_index()
test_proteins_long.head()
test_proteins_list = test_proteins_long.variable.values.tolist()
test_proteins_list=pd.unique(test_proteins_list).tolist()

In [48]:
test_proteins_list.insert(0,'updrs_2')
df_updrs_2_model=df_updrs_2[df_updrs_2.columns.intersection(test_proteins_list)]
print(df_updrs_2_model.shape)
df_updrs_2_model.head()

(1068, 93)


,updrs_2,O14773,O94919,P00736,P00738,P00748,P01009,P01011,P01019,P01033,...,Q6UXD5,Q7Z5P9,Q92823,Q92876,Q99969,Q9BY67,Q9UBR2,Q9UKV8,Q9UNU6,Q9Y6R7
0,6.0,31238.00,67567.6,109541.0,3956470.0,71835.4,14415900.0,2025890.0,1984650.0,117641.0,...,31029.1,519708.0,200896.0,1282260.0,22021.6,35217.8,23122.5,67571.7,29758.8,18953.5
2,10.0,26219.90,53314.5,105767.0,3944380.0,44460.6,13330800.0,2060930.0,1763540.0,128553.0,...,34010.2,444476.0,186814.0,1204530.0,23109.0,36506.1,23499.8,67571.7,22935.2,16642.7
4,10.0,30703.60,63075.8,112909.0,4107440.0,46942.3,13225500.0,2015710.0,2052090.0,138784.0,...,35116.9,462390.0,228730.0,1564970.0,22730.6,34450.3,21860.1,65762.6,29193.4,19290.9
8,18.0,33577.60,61609.3,98673.4,4063160.0,73678.9,13855000.0,2221880.0,2328900.0,167034.0,...,31733.0,487044.0,215727.0,1736340.0,23715.0,29545.9,23930.6,74976.1,31732.6,21717.1
15,2.0,6173.58,10549.7,18608.6,3038860.0,101312.0,13667800.0,2188150.0,1231570.0,38014.4,...,37863.3,467690.0,115232.0,676265.0,13060.3,24243.2,17450.9,82335.5,24018.7,15251.2


In [49]:
X=df_updrs_2_model.drop('updrs_2', axis=1)
y=df_updrs_2_model.iloc[:,0]

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=30
)

# Scale train and test sets with StandardScaler
#X_train_std = StandardScaler().fit_transform(X_train)
#X_test_std = StandardScaler().fit_transform(X_test)

# Fix the dimensions of the target array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

X_train.head()

,O14773,O94919,P00736,P00738,P00748,P01009,P01011,P01019,P01033,P01034,...,Q6UXD5,Q7Z5P9,Q92823,Q92876,Q99969,Q9BY67,Q9UBR2,Q9UKV8,Q9UNU6,Q9Y6R7
2230,19431.40,49408.70,85028.0,8588820.0,32926.4,14815300.0,1932860.0,1382170.0,110789.0,11856800.0,...,48600.80,618752.0,127423.0,1273840.0,17794.7,23498.0,16575.0,155934.0,28498.3,29041.3
1018,12022.80,26260.90,105312.0,2323560.0,82343.5,11912300.0,1608350.0,1599250.0,77037.7,28163100.0,...,29788.65,324582.0,206476.0,988955.0,14686.4,38526.8,13185.8,66544.4,22296.1,21158.8
96,21546.30,9306.78,19937.8,2613540.0,100328.0,9167420.0,1190940.0,1183300.0,62056.5,4605420.0,...,18253.10,552337.0,57064.2,558369.0,10019.6,26259.4,13134.1,79332.3,22527.4,21960.5
1764,9467.41,19706.10,101670.0,1666810.0,43610.3,12329500.0,1526950.0,1302720.0,77839.3,16533300.0,...,38363.20,610096.0,135400.0,1049390.0,12017.5,37540.7,12093.2,67571.7,16710.5,19749.0
121,20998.10,58203.20,25587.3,1374560.0,99805.6,10873600.0,1641650.0,1961200.0,92210.1,24500300.0,...,36438.00,437085.0,183124.0,1304500.0,13593.9,23940.0,16502.2,67571.7,25188.3,16701.5


In [50]:
m = XGBRegressor()
m.fit(X_train, y_train)

feat_dict={}
for col, val in sorted(zip(X_train.columns, m.feature_importances_), key=lambda x:x[1],reverse=True):
    feat_dict[col]=val
    
updrs_2_feat_xgb_model = pd.DataFrame({'Feature':feat_dict.keys(),'Importance':feat_dict.values()})
updrs_2_feat_xgb_model.to_csv('updrs_2_xgb_model.csv') 

In [51]:
y_pred = m.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

MSE: 29.79
RMSE: 5.46


In [52]:
updrs_2_array = m.predict(test_proteins_wide)
#mse = mean_squared_error(y_test, y_pred)
#print("MSE: %.2f" % mse)
#print("RMSE: %.2f" % (mse**(1/2.0)))

In [53]:
updrs_2_array

array([2.3829737, 9.008322 ], dtype=float32)

In [54]:
updrs_2_pred = pd.DataFrame(data=updrs_2_array,
                           columns=['Score'])
print(updrs_2_pred)

updrs_2_pred.to_csv('updrs_2_pred.csv')

      Score
0  2.382974
1  9.008322


In [55]:
submission.at[5,'rating']= updrs_2_array[0]
submission.at[20,'rating']= updrs_2_array[1]

In [56]:
df_updrs_3=train_clinical_proteins
df_updrs_3=df_updrs_3.dropna(subset=['Q99435', 'Q99674', 'Q99683', 'Q99829', 'Q99832'])
df_updrs_3=df_updrs_3.drop(['visit_id','patient_id','visit_month','updrs_1','updrs_2','updrs_4'],axis=1)
df_updrs_3=df_updrs_3.fillna(proteins_wide.median())
df_updrs_3=df_updrs_3[df_updrs_3['updrs_3'].notna()]
print(df_updrs_3.shape)
df_updrs_3.head()

(1058, 228)


,updrs_3,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,15.0,11254.3,732430.0,39585.8,41526.9,31238.00,4202.71,177775.0,62898.2,333376.0,...,365475.0,35528.0,97005.6,23122.5,60912.6,408698.0,67571.7,29758.8,23833.7,18953.5
2,34.0,13163.6,630465.0,35220.8,41295.0,26219.90,4416.42,165638.0,62567.5,277833.0,...,405676.0,30332.6,109174.0,23499.8,51655.8,369870.0,67571.7,22935.2,17722.5,16642.7
4,41.0,15257.6,815083.0,41650.9,39763.3,30703.60,4343.60,151073.0,66963.1,332401.0,...,303953.0,43026.2,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9
8,51.0,13530.8,753832.0,43048.9,43503.6,33577.60,5367.06,101056.0,67588.6,317490.0,...,303597.0,48188.4,109794.0,23930.6,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1
15,21.0,11218.7,399518.0,20581.0,31290.9,6173.58,2564.37,160526.0,43423.1,165524.0,...,253373.0,27431.8,93796.7,17450.9,21299.1,306621.0,82335.5,24018.7,18939.5,15251.2


In [57]:
df_updrs_3['updrs_3'].isnull().values.any()
df_updrs_3[df_updrs_3['updrs_3'].isna()]

,updrs_3,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7


In [58]:
X=df_updrs_3.drop('updrs_3', axis=1)
y=df_updrs_3.iloc[:,0]

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=15
)

# Scale train and test sets with StandardScaler
#X_train_std = StandardScaler().fit_transform(X_train)
#X_test_std = StandardScaler().fit_transform(X_test)

# Fix the dimensions of the target array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

m = XGBRegressor()
m.fit(X, y)

feat_dict={}
for col, val in sorted(zip(X_train.columns, m.feature_importances_), key=lambda x:x[1],reverse=True):
    feat_dict[col]=val
    
updrs_3_xgb = pd.DataFrame({'Feature':feat_dict.keys(),'Importance':feat_dict.values()})

updrs_3_xgb.to_csv('updrs_3_xgb.csv')

In [59]:
# Find cumulative importance and create list of columns to model

updrs_3_xgb['Cum_importance']=updrs_3_xgb['Importance'].cumsum()
updrs_3_xgb

updrs_3_xgb_keep=updrs_3_xgb.drop(updrs_3_xgb[updrs_3_xgb.Cum_importance >=0.80].index)
updrs_3_xgb_keep

updrs_3_xgb_keep_list = updrs_3_xgb_keep.Feature.values.tolist()

In [60]:
# Apply updrs_3_xgb_keep_list to test_proteins as this df doesn't include all the proteins of interest

test_proteins_wide = pd.pivot(test_proteins,index=['visit_id','visit_month','patient_id','group_key','role'],columns='UniProt',values='NPX')
test_proteins_wide=test_proteins_wide.rename_axis(None, axis=1).reset_index()
test_proteins_wide=test_proteins_wide.drop(['visit_id','visit_month','patient_id','group_key','role'], axis=1)
test_proteins_wide=test_proteins_wide[test_proteins_wide.columns.intersection(updrs_3_xgb_keep_list)]
test_proteins_wide=test_proteins_wide.replace(np.nan,0)
print(test_proteins_wide.shape)
test_proteins_wide.head()

(2, 88)


,O15240,O75144,P00738,P00747,P01009,P01011,P01019,P01033,P01042,P01594,...,Q6UXD5,Q92823,Q96KN2,Q96S96,Q99674,Q99969,Q9NQ79,Q9NYU2,Q9UBX5,Q9Y6R7
0,223373.0,160410.0,5200440.0,361287.0,16205500.0,1708870.0,2324710.0,0.0,1010440.0,0.00,...,0.0,240527.0,1190480.0,34384.9,31705.1,47999.1,46411.8,111979.0,40179.3,14624.5
1,124344.0,119277.0,563640.0,287692.0,14412800.0,2065690.0,1669670.0,105886.0,667371.0,8084.89,...,15938.2,157497.0,1143980.0,26700.6,19975.7,78596.4,35489.7,98781.8,50208.1,42920.4


In [61]:
test_proteins_long=test_proteins_wide.melt(ignore_index=False).reset_index()
test_proteins_long.head()
test_proteins_list = test_proteins_long.variable.values.tolist()
test_proteins_list=pd.unique(test_proteins_list).tolist()

In [62]:
test_proteins_list.insert(0,'updrs_3')
df_updrs_3_model=df_updrs_3[df_updrs_3.columns.intersection(test_proteins_list)]
print(df_updrs_3_model.shape)
df_updrs_3_model.head()

(1058, 89)


,updrs_3,O15240,O75144,P00738,P00747,P01009,P01011,P01019,P01033,P01042,...,Q6UXD5,Q92823,Q96KN2,Q96S96,Q99674,Q99969,Q9NQ79,Q9NYU2,Q9UBX5,Q9Y6R7
0,15.0,177775.0,98746.3,3956470.0,347865.0,14415900.0,2025890.0,1984650.0,117641.0,1221530.0,...,31029.1,200896.0,633503.0,34868.9,34550.6,22021.6,35528.0,97005.6,60912.6,18953.5
2,34.0,165638.0,95184.0,3944380.0,364918.0,13330800.0,2060930.0,1763540.0,128553.0,1045710.0,...,34010.2,186814.0,622718.0,33386.8,31721.1,23109.0,30332.6,109174.0,51655.8,16642.7
4,41.0,151073.0,93159.8,4107440.0,352298.0,13225500.0,2015710.0,2052090.0,138784.0,1296470.0,...,35116.9,228730.0,713274.0,39880.0,29439.7,22730.6,43026.2,114921.0,61598.2,19290.9
8,51.0,101056.0,96991.1,4063160.0,359799.0,13855000.0,2221880.0,2328900.0,167034.0,1122800.0,...,31733.0,215727.0,692022.0,40086.4,33892.0,23715.0,48188.4,109794.0,70223.5,21717.1
15,21.0,160526.0,49192.5,3038860.0,383735.0,13667800.0,2188150.0,1231570.0,38014.4,1270710.0,...,37863.3,115232.0,452697.0,32527.9,14387.7,13060.3,27431.8,93796.7,21299.1,15251.2


In [63]:
X=df_updrs_3_model.drop('updrs_3', axis=1)
y=df_updrs_3_model.iloc[:,0]

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=30
)

# Scale train and test sets with StandardScaler
#X_train_std = StandardScaler().fit_transform(X_train)
#X_test_std = StandardScaler().fit_transform(X_test)

# Fix the dimensions of the target array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

X_train.head()

,O15240,O75144,P00738,P00747,P01009,P01011,P01019,P01033,P01042,P01594,...,Q6UXD5,Q92823,Q96KN2,Q96S96,Q99674,Q99969,Q9NQ79,Q9NYU2,Q9UBX5,Q9Y6R7
1109,70974.2,50150.0,1315440.0,371318.0,13053700.0,1576000.0,1361580.0,73842.5,1053670.0,12650.6,...,22204.20,111568.0,507640.0,32527.9,16580.8,11265.20,34270.3,96918.0,44767.90,8854.69
1177,123448.0,44896.3,2436520.0,211930.0,9214740.0,1649730.0,1635290.0,48772.5,820485.0,22753.7,...,19993.00,135433.0,464243.0,32527.9,22502.7,9630.47,33936.4,101979.0,35151.20,31696.80
1995,47074.0,79815.6,3969180.0,360323.0,14347400.0,1572660.0,2091970.0,102936.0,962158.0,12615.3,...,29788.65,151440.0,454547.0,48518.2,21785.0,9495.36,30894.4,94630.2,48023.40,6844.72
2058,22374.1,91684.8,6954610.0,190476.0,11858000.0,2162140.0,1812760.0,112527.0,442786.0,39698.1,...,3423.25,105678.0,398213.0,46915.3,21326.4,641.39,17995.0,108209.0,57113.80,17132.10
573,120697.0,80438.2,4253620.0,371509.0,9653650.0,981305.0,896234.0,97986.1,873858.0,12615.3,...,29788.65,54158.1,419536.0,32527.9,18672.0,12330.50,33936.4,109984.0,35137.65,26963.60


In [64]:
m = XGBRegressor()
m.fit(X_train, y_train)

feat_dict={}
for col, val in sorted(zip(X_train.columns, m.feature_importances_), key=lambda x:x[1],reverse=True):
    feat_dict[col]=val
    
updrs_3_feat_xgb_model = pd.DataFrame({'Feature':feat_dict.keys(),'Importance':feat_dict.values()})
updrs_3_feat_xgb_model.to_csv('updrs_3_xgb_model.csv')

In [65]:
y_pred = m.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

MSE: 170.71
RMSE: 13.07


In [66]:
updrs_3_array = m.predict(test_proteins_wide)
#mse = mean_squared_error(y_test, y_pred)
#print("MSE: %.2f" % mse)
#print("RMSE: %.2f" % (mse**(1/2.0)))

In [67]:
updrs_3_pred = pd.DataFrame(data=updrs_3_array,
                           columns=['Score'])
print(updrs_3_pred)

updrs_3_pred.to_csv('updrs_3_pred.csv')

       Score
0  14.176371
1  27.185167


In [68]:
submission.at[9,'rating']= updrs_3_array[0]
submission.at[24,'rating']= updrs_3_array[1]

In [69]:
df_updrs_4=train_clinical_proteins
df_updrs_4=df_updrs_4.dropna(subset=['Q99435', 'Q99674', 'Q99683', 'Q99829', 'Q99832'])
df_updrs_4=df_updrs_4.drop(['visit_id','patient_id','visit_month','updrs_1','updrs_2','updrs_3'],axis=1)
df_updrs_4=df_updrs_4.fillna(proteins_wide.median())
df_updrs_4=df_updrs_4[df_updrs_4['updrs_4'].notna()]
print(df_updrs_4.shape)
df_updrs_4.head()

(569, 228)


,updrs_4,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
4,0.0,15257.60,815083.0,41650.9,39763.3,30703.6,4343.60,151073.0,66963.1,332401.0,...,303953.0,43026.2,114921.0,21860.1,61598.2,318553.0,65762.6,29193.4,28536.1,19290.9
8,0.0,13530.80,753832.0,43048.9,43503.6,33577.6,5367.06,101056.0,67588.6,317490.0,...,303597.0,48188.4,109794.0,23930.6,70223.5,377550.0,74976.1,31732.6,22186.5,21717.1
16,0.0,6757.32,360858.0,18367.6,14760.7,18603.4,1722.77,86847.4,37741.3,212132.0,...,45742.3,33518.6,94049.7,13415.7,21324.7,234094.0,82410.4,19183.7,17804.1,12277.0
22,0.0,11627.80,251820.0,22046.5,26360.5,22440.2,2117.43,82241.9,30146.6,167633.0,...,197987.0,29283.8,121696.0,19169.8,16724.9,232301.0,96905.9,21120.9,14089.8,16418.5
28,5.0,11648.90,419015.0,35053.8,26624.0,17466.3,2784.40,118752.0,40274.9,166322.0,...,306280.0,24931.3,109984.0,12610.9,40018.9,164826.0,55379.8,31563.0,19771.6,14699.5


In [70]:
X=df_updrs_4.drop('updrs_4', axis=1)
y=df_updrs_4.iloc[:,0]

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=15
)

# Scale train and test sets with StandardScaler
#X_train_std = StandardScaler().fit_transform(X_train)
#X_test_std = StandardScaler().fit_transform(X_test)

# Fix the dimensions of the target array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

m = XGBRegressor()
m.fit(X, y)

feat_dict={}
for col, val in sorted(zip(X_train.columns, m.feature_importances_), key=lambda x:x[1],reverse=True):
    feat_dict[col]=val
    
updrs_4_xgb = pd.DataFrame({'Feature':feat_dict.keys(),'Importance':feat_dict.values()})

updrs_4_xgb.to_csv('updrs_4_xgb.csv')

In [71]:
# Find cumulative importance and create list of columns to model

updrs_4_xgb['Cum_importance']=updrs_4_xgb['Importance'].cumsum()
updrs_4_xgb

updrs_4_xgb_keep=updrs_4_xgb.drop(updrs_4_xgb[updrs_4_xgb.Cum_importance >=0.80].index)
updrs_4_xgb_keep

updrs_4_xgb_keep_list = updrs_4_xgb_keep.Feature.values.tolist()

In [72]:
# Apply updrs_4_xgb_keep_list to test_proteins as this df doesn't include all the proteins of interest

test_proteins_wide = pd.pivot(test_proteins,index=['visit_id','visit_month','patient_id','group_key','role'],columns='UniProt',values='NPX')
test_proteins_wide=test_proteins_wide.rename_axis(None, axis=1).reset_index()
test_proteins_wide=test_proteins_wide.drop(['visit_id','visit_month','patient_id','group_key','role'], axis=1)
test_proteins_wide=test_proteins_wide[test_proteins_wide.columns.intersection(updrs_4_xgb_keep_list)]
test_proteins_wide=test_proteins_wide.replace(np.nan,0)
print(test_proteins_wide.shape)
test_proteins_wide.head()

(2, 60)


,O15394,P00736,P01009,P01023,P01031,P01344,P01608,P01621,P01780,P01857,...,Q16610,Q562R1,Q6UXB8,Q8IWV7,Q8NBJ4,Q8NE71,Q92876,Q96PD5,Q9NYU2,Q9UBX5
0,58863.0,115974.0,16205500.0,1846170.0,11004.3,87698.2,9509.49,134818.0,18883.7,5598370.0,...,280747.0,13941.5,42463.6,89461.1,26572.0,0.0,1364170.0,95103.2,111979.0,40179.3
1,57688.1,106727.0,14412800.0,1495280.0,10003.1,169379.0,16264.70,155036.0,0.0,4911390.0,...,244376.0,19601.1,67112.4,208404.0,21623.4,14768.7,1457830.0,108316.0,98781.8,50208.1


In [73]:
test_proteins_long=test_proteins_wide.melt(ignore_index=False).reset_index()
test_proteins_long.head()
test_proteins_list = test_proteins_long.variable.values.tolist()
test_proteins_list=pd.unique(test_proteins_list).tolist()

In [74]:
test_proteins_list.insert(0,'updrs_4')
df_updrs_4_model=df_updrs_4[df_updrs_4.columns.intersection(test_proteins_list)]
print(df_updrs_4_model.shape)
df_updrs_4_model.head()

(569, 61)


,updrs_4,O15394,P00736,P01009,P01023,P01031,P01344,P01608,P01621,P01780,...,Q16610,Q562R1,Q6UXB8,Q8IWV7,Q8NBJ4,Q8NE71,Q92876,Q96PD5,Q9NYU2,Q9UBX5
4,0.0,66963.1,112909.0,13225500.0,2007630.0,14103.50,128509.0,8539.53,99258.6,10916.30,...,228853.0,33830.20,60492.0,152944.0,34352.4,34246.7,1564970.0,128476.0,114921.0,61598.2
8,0.0,67588.6,98673.4,13855000.0,2279010.0,14416.00,142984.0,10201.80,86730.0,9063.14,...,224095.0,32804.70,56572.5,177998.0,28425.8,24592.0,1736340.0,112736.0,109794.0,70223.5
16,0.0,37741.3,51718.9,10101100.0,1083670.0,9459.22,61399.4,11471.30,217998.0,20733.00,...,131693.0,17542.90,26301.0,39817.9,20279.1,16899.2,688883.0,98935.9,94049.7,21324.7
22,0.0,30146.6,49750.5,11988000.0,1092670.0,12550.50,105089.0,16954.50,258267.0,17911.30,...,137706.0,17542.90,44500.8,141511.0,17533.9,15432.5,633725.0,123775.0,121696.0,16724.9
28,5.0,40274.9,65228.4,13234200.0,1118830.0,10950.20,73764.2,13594.70,170070.0,7679.01,...,141748.0,9334.67,37893.9,192593.0,23547.3,24274.5,1078780.0,72105.1,109984.0,40018.9


In [75]:
X=df_updrs_4_model.drop('updrs_4', axis=1)
y=df_updrs_4_model.iloc[:,0]

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=30
)

# Scale train and test sets with StandardScaler
#X_train_std = StandardScaler().fit_transform(X_train)
#X_test_std = StandardScaler().fit_transform(X_test)

# Fix the dimensions of the target array
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

X_train.head()

,O15394,P00736,P01009,P01023,P01031,P01344,P01608,P01621,P01780,P01857,...,Q16610,Q562R1,Q6UXB8,Q8IWV7,Q8NBJ4,Q8NE71,Q92876,Q96PD5,Q9NYU2,Q9UBX5
487,36257.1,51403.0,13892800.0,1436840.0,8802.68,126437.0,8486.41,150209.0,11278.60,3116630.0,...,142692.0,17542.9,48964.3,34092.0,20238.60,17032.40,1133820.0,53769.7,94918.3,30094.50
1839,39356.4,92187.2,13626600.0,1444320.0,11375.90,118433.0,13116.80,186596.0,2051.66,5422460.0,...,198589.0,23438.9,63833.7,152009.0,23089.05,9811.69,1032590.0,88753.3,78613.3,4184.43
22,30146.6,49750.5,11988000.0,1092670.0,12550.50,105089.0,16954.50,258267.0,17911.30,3825150.0,...,137706.0,17542.9,44500.8,141511.0,17533.90,15432.50,633725.0,123775.0,121696.0,16724.90
1942,47965.0,63095.2,11095100.0,1659390.0,11509.80,107905.0,16198.00,159785.0,11256.20,5872640.0,...,190493.0,15392.7,42351.7,178316.0,27336.10,17154.10,935370.0,103937.0,98073.4,36916.10
252,34550.1,62785.6,8262410.0,943733.0,11803.20,55768.9,12066.20,168293.0,11278.60,4303820.0,...,143209.0,17542.9,44653.1,133666.0,14074.60,12456.10,617176.0,123241.0,109984.0,15020.20


In [76]:
m = XGBRegressor()
m.fit(X_train, y_train)

feat_dict={}
for col, val in sorted(zip(X_train.columns, m.feature_importances_), key=lambda x:x[1],reverse=True):
    feat_dict[col]=val
    
updrs_4_feat_xgb_model = pd.DataFrame({'Feature':feat_dict.keys(),'Importance':feat_dict.values()})
updrs_4_feat_xgb_model.to_csv('updrs_4_xgb_model.csv')   

In [77]:
y_pred = m.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

MSE: 8.67
RMSE: 2.94


In [78]:
updrs_4_test = pd.DataFrame(data=y_pred,
                           columns=['Updrs_4'])

print(updrs_4_test)

updrs_4_test.to_csv('updrs_4_test.csv')

      Updrs_4
0    2.292405
1    3.619658
2    1.044420
3    2.786283
4    0.258215
..        ...
166  2.803881
167  1.835248
168  3.953099
169  1.457895
170  5.171997

[171 rows x 1 columns]


In [79]:
updrs_4_array = m.predict(test_proteins_wide)
#mse = mean_squared_error(y_test, y_pred)
#print("MSE: %.2f" % mse)
#print("RMSE: %.2f" % (mse**(1/2.0)))

In [80]:
updrs_4_pred = pd.DataFrame(data=updrs_4_array,
                           columns=['Score'])

print(updrs_4_pred)

updrs_4_pred.to_csv('updrs_4_pred.csv')

      Score
0  1.778984
1  2.293132


In [81]:
submission.at[13,'rating']= updrs_4_array[0]
submission.at[28,'rating']= updrs_4_array[1]

In [82]:
submission

,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,0.000000,0
1,3342_0_updrs_1_plus_6_months,4.287478,0
2,3342_0_updrs_1_plus_12_months,0.000000,0
3,3342_0_updrs_1_plus_24_months,0.000000,0
4,3342_0_updrs_2_plus_0_months,0.000000,0
...,...,...,...
59,50423_6_updrs_3_plus_24_months,0.000000,6
60,50423_6_updrs_4_plus_0_months,0.000000,6
61,50423_6_updrs_4_plus_6_months,0.000000,6
62,50423_6_updrs_4_plus_12_months,0.000000,6


In [83]:
sample_submission=submission[['prediction_id','rating']]

In [88]:
sys.path.append('../data/amp-parkinsons-disease-progression-prediction/')
import amp_pd_peptide

env = amp_pd_peptide.make_env()
iter_test = env.iter_test()

ModuleNotFoundError: No module named 'amp_pd_peptide.competition'

In [ ]:
counter = 0
# The API will deliver four dataframes in this specific order:
for (test, test_peptides, test_proteins, submission) in iter_test:
    if counter == 0:
        print(test.head())
        print(test_peptides.head())
        print(test_proteins.head())
        print(sample_submission.head(20))
        sample_submission['rating'] = 0
    env.predict(submission)
    counter += 1

In [ ]:
# target to prediction is derived from the mean of the mean for each Updrs prediction above

# target_to_prediciton = {
#    'updrs_1': 3.71, 
#    'updrs_2': 5.70, 
#    'updrs_3': 20.68, 
#    'updrs_4': 2.04
#}

In [ ]:
import amp_pd_peptide

amp_pd_peptide.make_env.func_dict['__called__'] = False
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files

# The API will deliver four dataframes in this specific order:

#iteration_to_data = {}
#for iteration, (test_clinical, test_peptides, test_proteins, sample_submission) in enumerate(iter_test):
#    print('ITERATION', iteration)
#    sample_submission['patient_id'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[0]))
#    sample_submission['visit_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[1]))
#    sample_submission['target_name'] = sample_submission['prediction_id'].map(lambda x: 'updrs_' + x.split('_')[3])
#    sample_submission['plus_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[5]))
#    sample_submission['pred_month'] = sample_submission['visit_month'] + sample_submission['plus_month']
#    data = {
#        'test_clinical': test_clinical,
#        'test_peptides': test_peptides,
#        'test_proteins': test_proteins,
#        'sample_submission': sample_submission
#    }
#    iteration_to_data[iteration] = data
    
#    print(test_clinical.shape)
#    display(test_clinical.head())
#    display(test_peptides.head())
#    display(test_proteins.head())
#    display(sample_submission.head())
#    print()
#    print()
    
    # change rating
#    for i in range(1, 5):
#        target = f'updrs_{i}'
#        mask_target = sample_submission['target_name'] == target
#        sample_submission.loc[mask_target, 'rating'] = target_to_prediciton[target]
    
#    # don't add additional columns (other than prediction_id, rating) in env.predict, it fails after submitting notebook
#    env.predict(sample_submission[['prediction_id', 'rating']])  